In [252]:
# package imports
import numpy as np
from itertools import islice
from scipy.linalg import eig
from scipy.stats import norm
from sklearn.preprocessing import normalize

In [253]:
# get stationary dustribution of transition matrix
# from stack overflow
def get_stationary_distibution(state_transition_matrix: np.ndarray) -> np.ndarray:
    S, U = eig(state_transition_matrix.T)
    stationary = np.array(U[:, np.where(np.abs(S - 1.) < 1e-8)[0][0]].flat)
    stationary = stationary / np.sum(stationary)
    return stationary

In [254]:
def estimate_sequence(state_transition_matrix:np.ndarray, gaussian_params: np.ndarray, initial_state_probability:np.ndarray, observations:np.ndarray, state_count: int):
    observation_count = observations.shape[0]
    state_probability_matrix = np.zeros((state_count, observation_count))
    path = np.ndarray((state_count, observation_count-1), dtype=int)
    emission_matrix = norm(loc=gaussian_params[:,0].T, scale=gaussian_params[:,1].T).pdf(observations).T
    emission_matrix = normalize(emission_matrix[:,:], axis=0)
    state_probability_matrix[:,0] = np.log2(initial_state_probability) + np.log2(emission_matrix[:,0])
    for i in range(1, observation_count):
        for j in range(state_count):
            prob = state_probability_matrix[:,i-1] + np.log2(state_transition_matrix[:,j]) + np.log2(emission_matrix[j,i])
            path[j,i-1] = np.argmax(prob)
            state_probability_matrix[j,i] = max(prob)

    return path, state_probability_matrix, emission_matrix
    # for i in range(1, observation_count):
    # cond_prob = norm.pdf(observations[1], loc=gaussian_params[:,0], scale=gaussian_params[:,1])
    # state_probability_matrix[0,1] = max(state_probability_matrix[:,0] + np.log2(transition_matrix[:,0]) + np.log2(cond_prob[0]))
    # state_probability_matrix[1,1] = max(state_probability_matrix[:,0] + np.log2(transition_matrix[:,1]) + np.log2(cond_prob[1]))
    # print(state_probability_matrix[:,1])
    # state_probability_matrix[:,1] = max(state_probability_matrix[:,0] + np.log2(transition_matrix) + np.log2(cond_prob))
    # print(state_probability_matrix[:,1])

In [255]:
# Viterbi algorithm
def viterbi():
    pass

In [256]:
# Baum-Welch Learning
def baum_welch():
    pass

In [257]:
# read data
observed_states = np.loadtxt('./Input/data.txt', dtype=float).reshape(-1,1)
observed_states.shape

(1000, 1)

In [258]:
# read parameters
with open('./Input/parameters.txt.txt', 'r') as f:
    no_of_states = int(f.readline())

with open('./Input/parameters.txt.txt', 'r') as lines:
    transition_matrix = np.genfromtxt(islice(lines, 1, 1+no_of_states))

with open('./Input/parameters.txt.txt', 'r') as lines:
    gaussian_parameters = np.genfromtxt(islice(lines, 1+no_of_states, 1+2*no_of_states), dtype=int)

gaussian_parameters

array([[200, 100],
       [ 10,  10]])

In [259]:
initial_distribution = get_stationary_distibution(transition_matrix)
initial_distribution

array([0.25, 0.75])

In [260]:
a, b, c = estimate_sequence(transition_matrix, gaussian_parameters, initial_distribution, observed_states, no_of_states)

In [261]:
# norm.pdf(104.524317662043, loc=gaussian_parameters[:,0], scale=gaussian_parameters[:,1])
a

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [262]:
b

array([[  -2.        ,   -2.51457317,   -3.02914635, ..., -515.02945331,
        -515.54402648, -516.05859966],
       [ -60.88685205,  -55.90568943,  -60.22484859, ..., -651.90559682,
        -566.15221123, -570.0015221 ]])

In [263]:
c

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [6.25417355e-19, 1.97537380e-16, 1.41369196e-17, ...,
        1.45932459e-41, 1.35954989e-15, 1.34753197e-16]])